# Fitting
Click [here](https://datahub.berkeley.edu/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fberkeley-physics%2FPython-Tutorials&urlpath=tree%2FPython-Tutorials%2F3+-+Specific+topics%2FFitting.ipynb&branch=master) to open this notebook in the DataHub.

## Learning objectives
By the end of this tutorial, you will be able to:
- Perform a linear fit to given data
- Assess the quality of a fit and recognise overfitting
- Estimate errors on fitted parameters
- Perform a curve fit on given data

## Relevant documentation
- [Numpy `Polynomial.fit`](https://numpy.org/doc/stable/reference/generated/numpy.polynomial.polynomial.Polynomial.fit.html#numpy.polynomial.polynomial.Polynomial.fit): Polynomial least-squares fitting
- [Numpy `polyfit`](https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html): Polynomial least-squares fitting
- [Scipy `linalg.lstsq`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html#scipy.linalg.lstsq): Linear least-squares fitting
- [Scipy `stats.linregress`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html#scipy.stats.linregress): Linear regression with additional statistics
- [Scipy `optimize.curve_fit`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html): Curve fitting
- More in-depth reference notebook for Python fitting by UCB Physics dept ([GitHub](https://github.com/berkeley-physics/intro_python/blob/master/Intro%20to%20stats.ipynb), [DataHub](https://datahub.berkeley.edu/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fberkeley-physics%2Fintro_python&urlpath=tree%2Fintro_python%2FIntro%2520to%2520stats.ipynb&branch=master))

## Linear fitting
Suppose we are trying to determine how some measured variable $y$ depends on some controlled variable $x$ (known exactly). We take $N$ pieces of data, $(x_i, y_i)$ for $i=0,1,..N-1$. Our goal is to come up with a model $y=f(x)$ that "best" matches our observed data.

Suppose we obtain the following data, with $N=41$, and we estimate that the measurement error is $\sigma(y)=1$. If it helps, you can think of physical descriptions for $x$ and $y$, e.g. distance from a light source and light intensity.

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

x = np.array([0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 
              2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5, 3.75, 
              4.0, 4.25, 4.5, 4.75, 5.0, 5.25, 5.5, 5.75, 
              6.0, 6.25, 6.5, 6.75, 7.0, 7.25, 7.5, 7.75, 
              8.0, 8.25, 8.5, 8.75, 9.0, 9.25, 9.5, 9.75, 10.0])
y = np.array([14.04465406568869, 15.449319071538802, 10.558376121970571, 
              14.278331548503782, 10.61501390320247, 10.262241155997827, 
              8.504167901005609, 10.411908528010994, 7.847512554941644, 
              9.506787464728173, 9.220630570268757, 7.793131432300264, 
              6.087691385289228, 7.445168521982977, 6.469870213838691, 
              6.215343174470461, 6.805532094970474, 6.220468686474819, 
              7.410791669925687, 4.072784928937637, 7.29188157791312, 
              6.035345890059883, 3.7982069549950834, 4.3394808067260096, 
              4.846882649841318, 6.53585887860196, 5.921629389303938, 
              5.241347215226233, 5.408648914774243, 5.52809378405438, 
              4.370303594475934, 3.9145280932994524, 4.348357111387002, 
              4.426927369888961, 4.92512354542205, 3.937578737964811, 
              6.00608193768343, 4.065285484161255, 4.350912089679179, 
              5.266872570811298, 3.122423953673369])
N = x.size
yerr = 1

plt.errorbar(x,y, yerr=yerr, fmt="o", capsize=2)
plt.xlabel("$x$")
plt.ylabel("$y$")

Now we need to think of a model $f(x)$ to describe $y$. In general, we will use a family of parametrised models $f(x;\mu)$, and find the best value for the parameters $\mu$ (which can be a vector) that describes the observed $y_i$. A common strategy is to measure the discrepancy between the model and data in terms of a scalar function $L(\mu)$, and find the $\mu$ at which this discrepancy is minimised. A commonly used approach is to use the summed squares of the _residuals_ (the difference between the model and data at each data point), $$ L(\mu) = \sum_{i=0}^{N-1}\big(y_i-f(x_i;\mu)\big)^2,$$ which is called _least squares_. For more information on when this is the appropriate function to minimise, see the reference notebook linked in the documentation section at the start of the notebook.

A common choice for a model is a polynomial $f(x;\mu)=a+bx+cx^2+...$, where the parameters are the polynomial coefficients $\mu=(a,b,c...)$. In this case (as in any case where $f$ is linear in $\mu$), this optimisation problem reduces to a set of linear equations, which can be solved by inverting a matrix (see the reference notebook for more information). This is called _linear least squares_. There are several convenient implementations of this in NumPy and SciPy. The following performs a fit to a degree-1 polynomial $f=a+bx$ using the `polyfit` function in NumPy. (To do a non-polynomial linear fit, see the reference notebook.)

In [ ]:
mu_fit = np.polyfit(x, y, deg=1) 
#best-fit coefficients, highest power first
mu_fit

In [ ]:
x_mesh = np.linspace(0,10,1000)
#evaluate polynomial given by coefficients mu_fit (1st arg) on arguments x_mesh (2nd arg)
model_mesh = np.polyval(mu_fit, x_mesh) 

plt.plot(x_mesh, model_mesh, c="r", label="model")
plt.errorbar(x,y, yerr=yerr, fmt="o", capsize=2, label="data")
plt.legend(frameon=False)
plt.xlabel("$x$")
plt.ylabel("$y$")

Try increasing the order of the polynomial. Does the fit improve? Keep increasing the order till you are obviously fitting noise (you will see wild swings in the model between data points). This is known as _overfitting_. In general, the more expressive your model, the more noise (and signal) you are able to fit, and you trade bias for variance.

A good way to check for any systematic errors is to plot the residuals, i.e. plot $y_i-f(x_i;\mu)$ against $x_i$. A good fit would have them evenly distributed across the $x$-axis, and a bad fit would have significant $x$-dependence. Try plotting this below. Is this a good fit? Try changing the order of the polynomial.

In [ ]:
plt.errorbar(x ,y-np.polyval(mu_fit,x), yerr=yerr, fmt="o", capsize=2)
plt.axhline(0, c="r")
plt.xlabel("$x$")
plt.ylabel(r"$y-f(x,\mu)$")

## Estimating errors
Suppose we wanted to assess the accuracy of our best-fit parameters $\mu_0$. The common approach is to see how the squared residuals change with the parameter. In general, the loss function is $$L(\mu) = (\mu-\mu_0)^T C^{-1} (\mu-\mu_0)+\rm constant,$$ where $C$ is called the _covariance matrix_. The diagonal elements of $C$ correspond to (squares of) our error estimates for each parameter. While the way it is presented here might make this seem arbitrary, it well-defined statistically; to understand it, see the reference notebook.

Let's consider a one-parameter example. The squared residuals are $$L(\mu) = \frac{(\mu-\mu_0)^2}{\sigma^2}+...,$$ where $\sigma$ is the error on our parameter: as $\mu$ is changed from its best-fit value, the faster the errors increase, the more precise our fit is.

The values of the errors (rather than ratios between them) only make sense if the errors in $y_i$ are on the scale of 1, but `numpy.polyfit` automatically scales $C$ to correct for that. To obtain the covariance matrix using `polyfit`, pass something into the keyword argument `cov`. We can pass our error estimates for $y$ by providing inversely proportional weights `w=1/y_err`. (If we didn't have estimates for the $y$ error, the algorithm would estimate it using the residuals and scale $C$ accordingly. We can turn off this auto-scaling of the covariance matrix by passing `cov="unscaled"`.

In [ ]:
mu_fit, cov = np.polyfit(x, y, deg=1, w=np.ones(N)/yerr, cov="unscaled") 
cov

In [ ]:
for degree, param in enumerate(mu_fit[::-1]):
    err = cov[-degree-1,-degree-1]**0.5
    print("x^%d coefficient: %.3f ± %.3f"%(degree,param,err))

Try increasing the order of the polynomial. Do the errors improve? Are estimated errors an indicator of goodness-of-fit?

## Curve fitting
Suppose we wanted to do a non-linear fit. In this case we cannot obtain the answer by simply inverting a matrix and need to optimise numerically (see the tutorial on solving equations for more information). For this we can use `curve_fit`. We need to pass it the model function, defined as $f(x,\mu_1,\mu_2,...)$. For example, here is the model function for a linear model $f(x;a,b) = a+ bx$:

In [ ]:
def linear_model(x,a,b):
    return a + b*x

The function returns the best-fit parameter values and the covariance matrix as defined above. Since the model may be non-linear, the squared residuals is not in general quadratic, but the function returns a quadratic approximation (the Hessian at the minimum).

In [ ]:
from scipy.optimize import curve_fit

mu, cov = curve_fit(linear_model, x, y, sigma=np.ones(N)*yerr, absolute_sigma=True)
errors = np.sqrt(np.diag(cov))

print("a = %.3f ± %.3f"%(mu[0],errors[0]))
print("b = %.3f ± %.3f"%(mu[1],errors[1]))

plt.plot(x_mesh, linear_model(x_mesh, *mu), c="r", label="model")
plt.errorbar(x,y, yerr=yerr, fmt="o", capsize=2, label="data")
plt.legend(frameon=False)
plt.xlabel("$x$")
plt.ylabel("$y$")

Sometimes, we might have some models in mind we might want to try. For the example of the light source, we might imagine that the intensity drops exponentially with distance, leading to a model like $f(x) = Ae^{-Bx}$. There might also be a background light intensity (which in practice one could calibrate against by measuring the intensity with the light switched off), leading to the model $f(x)=Ae^{-Bx}+C$. 

Try fitting this model, or one of your own, to the data: find the best-fit parameters (with errors), plot the residuals, and plot the exponential fit with the data as well as your favourite polynomial fits.

By the way, the data was generated using $$y_i = 10 e^{-x_i/3}+4+n,$$ where $n$ is a normally distributed "random" variable. In the light-source example, the average background light intensity is 4 \[$y$ units\], the intensity of the light source is 10 \[$y$ units\], and the optical depth of the medium for this frequency ditribution is 3 \[$x$ units\]. How do the true values for the parameters compare with your estimates from the data? Add the true model to the plot with your best-fit model and the data. Does the true line lie inside all the errorbars? Why or why not?